In [1]:
import os
from dotenv import load_dotenv

load_dotenv()

os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
os.environ["LANGSMITH_API_KEY"] = os.getenv("LANGSMITH_API_KEY")  # langsmith tracking
os.environ["LANGSMITH_TRACING"] = os.getenv("LANGSMITH_TRACING")
os.environ["LANGSMITH_PROJECT"] = os.getenv("LANGSMITH_PROJECT")
os.environ["LANGSMITH_ENDPOINT"] = os.getenv("LANGSMITH_ENDPOINT")

In [6]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(
    api_key = os.environ["OPENAI_API_KEY"],
    model = "gpt-4o"
)

In [8]:
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage

In [9]:
model.invoke(
    [
        HumanMessage(content="Hello, world!"),
    ]
)

AIMessage(content='Hello! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 11, 'total_tokens': 21, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_85bc8e3fae', 'id': 'chatcmpl-BMp7ej2061C0tFnSBdMTLSzJLTTX3', 'finish_reason': 'stop', 'logprobs': None}, id='run-efef7aa7-eecd-4953-996c-b6eb6d6936ab-0', usage_metadata={'input_tokens': 11, 'output_tokens': 10, 'total_tokens': 21, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [10]:
model.invoke(
    [
        HumanMessage(content="Hello, world!"),
        AIMessage(content="Hello! How can I assist you today?"),
        HumanMessage(content="I need help with my homework.")
    ]
)

AIMessage(content='Of course! What do you need help with?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 11, 'prompt_tokens': 35, 'total_tokens': 46, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_85bc8e3fae', 'id': 'chatcmpl-BMp8JUQH64mcmVlZaNXBcyaKYgc1f', 'finish_reason': 'stop', 'logprobs': None}, id='run-c9d7c132-4632-4a4c-9807-68e993755070-0', usage_metadata={'input_tokens': 35, 'output_tokens': 11, 'total_tokens': 46, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [12]:
# Message History

from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory


In [13]:
store = {}
def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

In [14]:
config = {
    "configurable":{
        "session_id":"Chat_1"
    }
}

In [15]:
with_message_history = RunnableWithMessageHistory(
    model,
    get_session_history=get_session_history
)

In [17]:
response = with_message_history.invoke(
    [
        HumanMessage(
            content="What is the capital of France?",
        )
    ],
    config=config
)

In [18]:
response.content

'The capital of France is Paris.'

In [19]:
config_2 = {
    "configurable":{
        "session_id":"Chat_3"
    }
}
response = with_message_history.invoke(
    [
        HumanMessage(
            content="What is the capital of India?",
        )
    ],
    config=config_2
)

In [20]:
response.content


'The capital of India is New Delhi.'

In [24]:
## Prompt templating
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant."),
        MessagesPlaceholder(variable_name="messages")
    ]
)
chain = prompt|model

In [23]:
chain.invoke(
    {
        "messages":[
            HumanMessage(content="What is the weather like today?")
        ]
    }
)

ChatPromptTemplate(input_variables=['human'], input_types={'human': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages.chat.ChatMessageChunk, Tag(tag='ChatMessageChunk')], typing.Annotated[langchain_core.messages.system.SystemMessageChunk, Tag(tag='SystemMessageChunk')], typing.Annotated[langchain_core.messages.

In [26]:
with_message_history = RunnableWithMessageHistory(
    chain, get_session_history=get_session_history
)
config_3 = {
    "configurable":{
        "session_id":"Chat_2"
    }
}
response = with_message_history.invoke(
    [HumanMessage(content="What is the weather like today?")], config=config_3
)

In [40]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("system",
         "You are a helpful assistant. Answer in following language: {language}"),
        MessagesPlaceholder(variable_name="messages")
    ]
)
chain_2 = prompt|model

In [50]:
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history=get_session_history,
    input_messages_key="messages"  # <-- this is the fix
)


config_4 = {
    "configurable":{
        "session_id":"Chat_4"
    }
}
# response = chain_2.invoke({
#         "messages":[HumanMessage(content="Hi, ravenking this side")],
#         "language":"hindi"
#     }
# )

In [57]:
response = with_message_history.invoke(
    config=config_4,
    input={
        "messages": [
            HumanMessage(
                content="Hi how are you?",
            )
        ],
        "language": "Hindi"
    }
)


In [58]:
response.content

"Hello! I'm here and ready to assist you. How can I help you today?"